Ma proposition : 
Avoir un Notebook template associé à des paramètres par défaut. 
Ensuite pour chaque expérience on le duplique, et on change les valeurs des paramètres que l'on souhaite.

In [8]:
import yaml
with open('../configs/default_params.yaml') as default_params_file:
      default_params = yaml.safe_load(default_params_file)
params = default_params
params

{'experiment_title': 'write here the purpose of your experiment',
 'dataset': {'script_path': '../data_processing/A_COMPLETER.py'},
 'model': {'script_path': '../models/A_COMPLETER.py'},
 'optimizer': {'script_path': '../optimizers/adam_keras.py',
  'initial_lr': 0.0001},
 'train': {'script_path': '../train/A_COMPLETER.py',
  'batch_size': 64,
  'epochs': 1000,
  'data_augmentation': {'samplewise_center': False,
   'samplewise_std_normalization': False,
   'rotation_range': 0,
   'width_shift_range': 0.1,
   'height_shift_range': 0.1,
   'horizontal_flip': True,
   'vertical_flip': False,
   'zoom_range': 0,
   'shear_range': 0,
   'channel_shift_range': 0,
   'featurewise_center': False,
   'zca_whitening': False}},
 'evaluate': {'batch_size': 1000,
  'augmentation_factor': 32,
  'data_augmentation': {'samplewise_center': False,
   'samplewise_std_normalization': False,
   'rotation_range': 0,
   'width_shift_range': 0.15,
   'height_shift_range': 0.15,
   'horizontal_flip': True,
   

In [9]:
# Changer de paramètres
params["train"]["batch_size"] = 16
params["train"]

{'script_path': '../train/A_COMPLETER.py',
 'batch_size': 10,
 'epochs': 1000,
 'data_augmentation': {'samplewise_center': False,
  'samplewise_std_normalization': False,
  'rotation_range': 0,
  'width_shift_range': 0.1,
  'height_shift_range': 0.1,
  'horizontal_flip': True,
  'vertical_flip': False,
  'zoom_range': 0,
  'shear_range': 0,
  'channel_shift_range': 0,
  'featurewise_center': False,
  'zca_whitening': False}}